In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [2]:
#Download dữ liệu phân bón dried (bao gồm gia súc và gia cầm)
!gdown --id 1Jqb1nXv-fNqXGYxsShfSdvm0F2kabNhW
#Download dữ liệu phân bón fresh (bao gồm gia súc và gia cầm)
!gdown --id 1Us68Nzzafwr1VPsYnYv-cEJZBmNJ3Ji3
#Download dữ liệu thành phần hóa học của phân bón
!gdown --id 1qdONOOyIcmJfR1vZSaGzIvHtyTg7ShfP

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Jqb1nXv-fNqXGYxsShfSdvm0F2kabNhW
To: /content/spectra_DG_Abs_STD_1100_2498nm_STD.xlsx
100% 1.61M/1.61M [00:00<00:00, 211MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Us68Nzzafwr1VPsYnYv-cEJZBmNJ3Ji3
To: /content/spectra_FH_Abs_STD_1100_2498nm_STD.xlsx
100% 2.17M/2.17M [00:00<00:00, 156MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.

In [3]:
file_paths = [
    "/content/spectra_DG_Abs_STD_1100_2498nm_STD.xlsx",
    "/content/spectra_FH_Abs_STD_1100_2498nm_STD.xlsx",
    "/content/chemical_analysis.xlsx"
]

fresh_spectra_df = pd.read_excel(file_paths[1])
chemical_df = pd.read_excel(file_paths[2])

In [4]:
type_manure_df = chemical_df[['type_manure']]
fresh_df = pd.concat([fresh_spectra_df, type_manure_df], axis=1).drop(columns='sample_name')
fresh_df

,1100_nm,1102_nm,1104_nm,1106_nm,1108_nm,1110_nm,1112_nm,1114_nm,1116_nm,1118_nm,...,2482_nm,2484_nm,2486_nm,2488_nm,2490_nm,2492_nm,2494_nm,2496_nm,2498_nm,type_manure
0,0.588418,0.588407,0.588012,0.587727,0.587602,0.587279,0.586653,0.586186,0.586046,0.585861,...,1.686953,1.689275,1.691759,1.694338,1.696744,1.698854,1.700564,1.701765,1.702570,cattle manure
1,0.530671,0.530607,0.530356,0.530160,0.530084,0.529749,0.529146,0.528818,0.528807,0.528645,...,1.579084,1.581248,1.583488,1.585777,1.587941,1.589935,1.591674,1.593029,1.594062,cattle manure
2,0.532509,0.532652,0.532556,0.532679,0.532500,0.532282,0.531813,0.531593,0.531658,0.531550,...,1.612479,1.614203,1.616133,1.618259,1.620364,1.622323,1.623984,1.625191,1.626028,cattle manure
3,0.495200,0.494887,0.494335,0.493886,0.493594,0.493283,0.492497,0.492086,0.492000,0.491789,...,1.568502,1.570339,1.572338,1.574459,1.576464,1.578253,1.579747,1.580875,1.581749,cattle manure
4,0.476230,0.476494,0.476465,0.476356,0.476468,0.476366,0.476144,0.476265,0.476501,0.476709,...,1.603243,1.604913,1.606800,1.608894,1.610958,1.612836,1.614357,1.615386,1.616054,cattle manure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,0.921903,0.921882,0.921879,0.921892,0.921893,0.921852,0.921772,0.921675,0.921602,0.921573,...,1.525306,1.528573,1.531451,1.534228,1.537057,1.539675,1.541941,1.544031,1.546002,cattle manure
328,1.226104,1.226319,1.226631,1.226985,1.227300,1.227568,1.227816,1.228038,1.228266,1.228530,...,1.560118,1.562692,1.565166,1.567635,1.570050,1.572208,1.574032,1.575824,1.577748,cattle manure
329,1.208262,1.208337,1.208493,1.208694,1.208869,1.208999,1.209085,1.209125,1.209165,1.209258,...,1.786263,1.790263,1.793536,1.796581,1.799726,1.802921,1.805966,1.809047,1.812130,cattle manure
330,0.571057,0.570358,0.569508,0.568678,0.567928,0.567190,0.566426,0.565650,0.564924,0.564303,...,1.550831,1.553917,1.556775,1.559369,1.561662,1.563666,1.565651,1.567573,1.569299,cattle manure


In [5]:
#poultry manure: 181 -> 316
#cattle manure: 1 -> 180 ; 317-> 332

Class = fresh_df[['type_manure']].values
# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Thực hiện mã hóa nhãn
encoded_classes = label_encoder.fit_transform(Class)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
scaler = StandardScaler()
#scaler = MinMaxScaler()
X = scaler.fit_transform(fresh_df.drop(columns='type_manure'))
y = encoded_classes
print(X.shape, y.shape)

(332, 700) (332,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

In [8]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
import numpy as np
import pandas as pd

In [9]:
class cGAN():
    def __init__(self):
        self.latent_dim = 32
        self.out_shape = 29
        self.num_classes = 2
        self.clip_value = 0.01
        optimizer = Adam(0.0002, 0.5)
        #optimizer = RMSprop(lr=0.00005)

        # build discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # build generator
        self.generator = self.build_generator()

        # generating new data samples
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        gen_samples = self.generator([noise, label])

        self.discriminator.trainable = False

        # passing gen samples through disc.
        valid = self.discriminator([gen_samples, label])

        # combining both models
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
                              optimizer=optimizer,
                             metrics=['accuracy'])
        self.combined.summary()

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(128, input_dim=self.latent_dim))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(256))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        #model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(self.out_shape, activation='tanh'))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        gen_sample = model(model_input)

        return Model([noise, label], gen_sample, name="Generator")


    def build_discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(512, input_dim=self.out_shape, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(128, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))

        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        gen_sample = Input(shape=(self.out_shape,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.out_shape)(label))

        model_input = multiply([gen_sample, label_embedding])
        validity = model(model_input)

        return Model(inputs=[gen_sample, label], outputs=validity, name="Discriminator")


    def train(self, X_train, y_train, pos_index, neg_index, epochs, batch_size=32, sample_interval=50):

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            #  Train Discriminator with 8 sample from postivite class and rest with negative class
            idx1 = np.random.choice(pos_index, 8)
            idx0 = np.random.choice(neg_index, batch_size-8)
            idx = np.concatenate((idx1, idx0))
            samples, labels = X_train[idx], y_train[idx]
            samples, labels = shuffle(samples, labels)
            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a half batch of new images
            gen_samples = self.generator.predict([noise, labels])

            # label smoothing
            if epoch < epochs//1.5:
                valid_smooth = (valid+0.1)-(np.random.random(valid.shape)*0.1)
                fake_smooth = (fake-0.1)+(np.random.random(fake.shape)*0.1)
            else:
                valid_smooth = valid
                fake_smooth = fake

            # Train the discriminator
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch([samples, labels], valid_smooth)
            d_loss_fake = self.discriminator.train_on_batch([gen_samples, labels], fake_smooth)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            # Condition on labels
            self.discriminator.trainable = False
            sampled_labels = np.random.randint(0, 2, batch_size).reshape(-1, 1)
            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            if (epoch+1)%sample_interval==0:
                print (f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

In [10]:
cgan = cGAN()

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               15360     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 128)               0

In [11]:
y_train = y_train.reshape(-1,1)
pos_index = np.where(y_train==1)[0]
neg_index = np.where(y_train==0)[0]
cgan.train(X_train, y_train, pos_index, neg_index, epochs=2000)

1/1 [==============================] - 1s 508ms/step


ValueError: ignored

In [ ]:
# generating new samples
# noise = np.random.normal(0, 1, (400, 32))
# sampled_labels = np.ones(400).reshape(-1, 1)

# gen_samples = cgan.generator.predict([noise, sampled_labels])
# gen_samples = scaler.inverse_transform(gen_samples)
# print(gen_samples.shape)

In [ ]:
# gen_df = pd.DataFrame(data = gen_samples, columns = fresh_df.drop('type_manure').columns)
# gen_df.head()